In [1]:
# 1️⃣ استيراد المكتبات
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import VotingClassifier
import joblib

In [14]:
import pandas as pd

file_path = "cumulative_2025.09.30_12.52.38 (3).csv"

try:
    # تجاهل الأسطر التي تبدأ بـ # (تعليقات)
    tabular_df = pd.read_csv(file_path, comment='#', low_memory=False)
    print("تم تحميل الملف بنجاح!")
    print(tabular_df.head())
except Exception as e:
    print("حدث خطأ أثناء قراءة الملف:", e)


تم تحميل الملف بنجاح!
      kepid kepoi_name   kepler_name koi_disposition koi_pdisposition  \
0  10797460  K00752.01  Kepler-227 b       CONFIRMED        CANDIDATE   
1  10797460  K00752.02  Kepler-227 c       CONFIRMED        CANDIDATE   
2  10811496  K00753.01           NaN       CANDIDATE        CANDIDATE   
3  10848459  K00754.01           NaN  FALSE POSITIVE   FALSE POSITIVE   
4  10854555  K00755.01  Kepler-664 b       CONFIRMED        CANDIDATE   

   koi_score  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  ...  \
0      1.000              0              0              0              0  ...   
1      0.969              0              0              0              0  ...   
2      0.000              0              0              0              0  ...   
3      0.000              0              1              0              0  ...   
4      1.000              0              0              0              0  ...   

   koi_steff_err2  koi_slogg  koi_slogg_err1  koi_sl

In [15]:
print(tabular_df.columns.tolist()) #فحص الاعمدة


['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'koi_kepmag']


In [18]:
features = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg','koi_srad','koi_kepmag','koi_disposition']
tabular_df = tabular_df[features]


In [19]:
# قائمة الأعمدة المتوفرة فقط
available_features = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg','koi_srad','koi_kepmag']
features = available_features.copy()


In [21]:
from sklearn.impute import SimpleImputer

# استخدام median لتعويض القيم المفقودة
imputer = SimpleImputer(strategy='median')
tabular_df[features] = imputer.fit_transform(tabular_df[features])


In [22]:
X_tab = tabular_df[features].values #تجهيز X (الخصائص) و y (الفئة)

y = tabular_df['koi_disposition'].values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [23]:
from sklearn.preprocessing import StandardScaler #StandardScaler للخصائص
scaler_tab = StandardScaler()
X_tab_scaled = scaler_tab.fit_transform(X_tab)


In [24]:
# 4️⃣ تقسيم بيانات الفيزياء
X_tab_train, X_tab_test, y_train, y_test = train_test_split(X_tab_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [26]:
#5بناء نموذج XGBoost
xgb_model = XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', num_class=len(np.unique(y_encoded)), use_label_encoder=False, random_state=42)
xgb_model.fit(X_tab_train, y_train)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [20:06:24] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3, ...)

In [30]:
import pandas as pd
import ast

# افترض أن الملف مرفوع باسم file_name
lines = open(file_name).readlines()

# البحث عن الصفوف الفعلية (تجاهل أي تعليقات في الأعلى)
# افترض أن العمود الأول هو flux_values والثاني koi_disposition
data = []
for line in lines[1:]:  # تجاهل الهيدر إذا موجود
    try:
        flux_str, disposition = line.strip().split(',', 1)  # نفصل عند أول فاصلة
        flux_list = ast.literal_eval(flux_str)  # تحويل النص لقائمة
        data.append([flux_list, disposition])
    except:
        continue  # تجاهل أي صفوف تالفة

# تحويل إلى DataFrame
lightcurve_df = pd.DataFrame(data, columns=['flux_values','koi_disposition'])
print(lightcurve_df.head())


   flux_values                                    koi_disposition
0     10797460  K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.0...
1     10797460  K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.9...
2     10811496  K00753.01,,CANDIDATE,CANDIDATE,0.0000,0,0,0,0,...
3     10848459  K00754.01,,FALSE POSITIVE,FALSE POSITIVE,0.000...
4     10854555  K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.0...


In [32]:
# التأكد أن كل عنصر في flux_values هو قائمة
def ensure_list(x):
    if isinstance(x, list):
        return x
    else:
        return [x]  # تحويل أي رقم مفرد لقائمة تحتويه

lightcurve_df['flux_values'] = lightcurve_df['flux_values'].apply(ensure_list)

# إزالة أي صفوف فارغة أو تحتوي على None
lightcurve_df = lightcurve_df[lightcurve_df['flux_values'].apply(lambda x: len(x) > 0)]

# توحيد طول السلاسل
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 200
X_lc = pad_sequences(lightcurve_df['flux_values'], maxlen=MAX_LEN, dtype='float32', padding='post', truncating='post')

# فحص الناتج
print(X_lc.shape)


(9564, 200)


In [35]:
# لنفترض أن لدينا عمود نصي يحتوي كل CSV في صف واحد
# نفصل النص عند الفاصلة ونأخذ العمود الصحيح للفئة (عادة 3rd column)
def extract_disposition(row):
    try:
        return row.split(',')[2].strip()
    except:
        return None

lightcurve_df['koi_disposition_clean'] = lightcurve_df['koi_disposition'].apply(extract_disposition)

# إزالة أي صفوف غير صالحة
lightcurve_df = lightcurve_df[lightcurve_df['koi_disposition_clean'].notna()]


In [36]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_lc = le.fit_transform(lightcurve_df['koi_disposition_clean'].values)

from tensorflow.keras.utils import to_categorical
y_lc_cat = to_categorical(y_lc)


In [37]:
print(X_lc.shape, y_lc_cat.shape)


(9564, 200) (9564, 3)


In [38]:

# تقسيم بيانات Light Curve
X_lc_train, X_lc_test, y_lc_train, y_lc_test = train_test_split(X_lc, y_lc_cat, test_size=0.2, random_state=42, stratify=y_lc_cat)

In [39]:

# Reshape لتتناسب مع Conv1D
X_lc_train = X_lc_train.reshape((X_lc_train.shape[0], X_lc_train.shape[1], 1))
X_lc_test = X_lc_test.reshape((X_lc_test.shape[0], X_lc_test.shape[1], 1))

In [40]:

# 6️⃣ بناء نموذج LSTM+1D-CNN
lc_input = Input(shape=(MAX_LEN,1))
x = Conv1D(64,5,activation='relu')(lc_input)
x = MaxPooling1D(2)(x)
x = LSTM(64)(x)
x = Dropout(0.3)(x)
x = Dense(32,activation='relu')(x)
lc_output = Dense(y_lc_cat.shape[1], activation='softmax')(x)
lc_model = Model(inputs=lc_input, outputs=lc_output)
lc_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lc_model.fit(X_lc_train, y_lc_train, validation_split=0.2, epochs=50, batch_size=32, callbacks=[early_stop], verbose=1)

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.4938 - loss: 1.0448 - val_accuracy: 0.4977 - val_loss: 1.0388
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.4976 - loss: 1.0388 - val_accuracy: 0.4977 - val_loss: 1.0368
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.5078 - loss: 1.0298 - val_accuracy: 0.4977 - val_loss: 1.0334
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.5029 - loss: 1.0352 - val_accuracy: 0.4977 - val_loss: 1.0328
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.5062 - loss: 1.0323 - val_accuracy: 0.4977 - val_loss: 1.0309
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.5059 - loss: 1.0307 - val_accuracy: 0.4977 - val_loss: 1.0317
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.5197 - loss: 1.0203 - val_accuracy: 0.4977 - val_loss: 1.0313
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.5117 - loss: 1.0275 - val_a

In [42]:
# 7️⃣ دمج النتائج (Hybrid Ensemble)
# نحصل على الاحتمالات لكل نموذج ثم ندمجها (Soft Voting)
xgb_probs = xgb_model.predict_proba(X_tab_test)
lc_probs = lc_model.predict(X_lc_test)

# إذا كانت أطوال الاختبار متطابقة، ندمج الاحتمالات
hybrid_probs = (xgb_probs + lc_probs) / 2
hybrid_pred = np.argmax(hybrid_probs, axis=1)

60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [44]:
# 8️⃣ Evaluate Hybrid Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, hybrid_pred))

print("\nClassification Report:")
print(classification_report(y_test, hybrid_pred, target_names=le.classes_))

Confusion Matrix:
[[129  77 190]
 [ 37 443  69]
 [ 38  53 877]]

Classification Report:
                precision    recall  f1-score   support

     CANDIDATE       0.63      0.33      0.43       396
     CONFIRMED       0.77      0.81      0.79       549
FALSE POSITIVE       0.77      0.91      0.83       968

      accuracy                           0.76      1913
     macro avg       0.73      0.68      0.68      1913
  weighted avg       0.74      0.76      0.74      1913



In [61]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# دقة النموذج الكلي
acc = accuracy_score(y_test, hybrid_pred)
print("Accuracy:", acc)

# مصفوفة الالتباس
print("Confusion Matrix:")
print(confusion_matrix(y_test, hybrid_pred))

# تقرير التصنيف لكل فئة
print("\nClassification Report:")
print(classification_report(y_test, hybrid_pred, target_names=le.classes_))


Accuracy: 0.7574490329325666
Confusion Matrix:
[[129  77 190]
 [ 37 443  69]
 [ 38  53 877]]

Classification Report:
                precision    recall  f1-score   support

     CANDIDATE       0.63      0.33      0.43       396
     CONFIRMED       0.77      0.81      0.79       549
FALSE POSITIVE       0.77      0.91      0.83       968

      accuracy                           0.76      1913
     macro avg       0.73      0.68      0.68      1913
  weighted avg       0.74      0.76      0.74      1913



In [68]:
#هذا الكود لتحسين ال دقه لان الدقه 0.75
# 🔹 تثبيت Keras Tuner إذا لم يكن مثبت
!pip install keras-tuner -q

# 🔹 استيراد المكتبات
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib

# ==============================
# 1️⃣ تحميل البيانات الفيزيائية (tabular)
# ==============================
tabular_df = pd.read_csv("cumulative_2025.09.30_12.52.38 (3).csv", comment='#', low_memory=False)

# ==============================
# 2️⃣ Feature Engineering
# ==============================
features = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg','koi_srad','koi_kepmag']
tabular_df['radius_ratio'] = tabular_df['koi_prad'] / tabular_df['koi_srad']
tabular_df['orbital_density'] = tabular_df['koi_prad'] / tabular_df['koi_period']
features.extend(['radius_ratio','orbital_density'])

# ==============================
# 3️⃣ معالجة البيانات المفقودة + تجهيز X و y
# ==============================
imputer = SimpleImputer(strategy='median')
X_tab = imputer.fit_transform(tabular_df[features])

y = tabular_df['koi_disposition'].values
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# ==============================
# 4️⃣ تطبيع البيانات
# ==============================
scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

# ==============================
# 5️⃣ تقسيم البيانات
# ==============================
X_train, X_test, y_train, y_test = train_test_split(X_tab_scaled, y_cat, test_size=0.2, random_state=42, stratify=y_cat)

# ==============================
# 6️⃣ Keras Tuner: تعريف دالة بناء النموذج
# ==============================
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=256, step=32),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout1', 0.0, 0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units2', min_value=16, max_value=128, step=16),
                    activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout2', 0.0, 0.5, step=0.1)))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ==============================
# 7️⃣ تهيئة Tuner
# ==============================
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='hybrid_tuning',
    project_name='kepler'
)

# ==============================
# 8️⃣ Callbacks لتحسين التدريب
# ==============================
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# ==============================
# 9️⃣ البحث عن أفضل Hyperparameters
# ==============================
tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stop, reduce_lr])

# ==============================
# 10️⃣ استخراج أفضل نموذج
# ==============================
best_model = tuner.get_best_models(num_models=1)[0]

# ==============================
# 11️⃣ تقييم النموذج
# ==============================
y_pred_prob = best_model.predict(X_test)
y_pred_idx = np.argmax(y_pred_prob, axis=1)
y_test_idx = np.argmax(y_test, axis=1)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Accuracy:", accuracy_score(y_test_idx, y_pred_idx))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_idx, y_pred_idx))
print("\nClassification Report:\n", classification_report(y_test_idx, y_pred_idx, target_names=le.classes_))

# ==============================
# 12️⃣ حفظ النموذج والـ Scaler
# ==============================
best_model.save('hybrid_model_improved.h5')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(le, 'label_encoder.pkl')
print("✅ Hybrid model improved, scaler, and label encoder saved!")


Trial 5 Complete [00h 00m 16s]
val_accuracy: 0.6792945861816406

Best val_accuracy So Far: 0.6819072365760803
Total elapsed time: 00h 01m 11s
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Accuracy: 0.6795608991113434

Confusion Matrix:
 [[ 24 144 228]
 [ 13 440  96]
 [ 16 116 836]]

Classification Report:
                 precision    recall  f1-score   support

     CANDIDATE       0.45      0.06      0.11       396
     CONFIRMED       0.63      0.80      0.70       549
FALSE POSITIVE       0.72      0.86      0.79       968

      accuracy                           0.68      1913
     macro avg       0.60      0.58      0.53      1913
  weighted avg       0.64      0.68      0.62      1913

✅ Hybrid model improved, scaler, and label encoder saved!


In [69]:
from tensorflow.keras.models import Model #بناء Hybrid Model وتدريبه على البيانات التي حضرناها
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten, concatenate

# Inputs
input_tab = Input(shape=(X_tab_scaled.shape[1],), name='tabular_input')
x_tab = Dense(64, activation='relu')(input_tab)
x_tab = Dense(32, activation='relu')(x_tab)

input_lc = Input(shape=(X_lc.shape[1], 1), name='lc_input')
x_lc = LSTM(32)(input_lc)
x_lc = Dense(32, activation='relu')(x_lc)

# دمج الجزئين
x = concatenate([x_tab, x_lc])
output = Dense(3, activation='softmax')(x)

hybrid_model = Model(inputs=[input_tab, input_lc], outputs=output)
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# تدريب النموذج
hybrid_model.fit([X_tab_scaled, X_lc[..., np.newaxis]], y_lc_cat, epochs=10, batch_size=32)


Epoch 1/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - accuracy: 0.4934 - loss: 1.0014
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.5986 - loss: 0.9049
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.6207 - loss: 0.8538
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6219 - loss: 0.8245
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6371 - loss: 0.8146
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.6366 - loss: 0.7997
Epoch 7/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6393 - loss: 0.7870
Epoch 8/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6445 - loss: 0.7736
Epoch 9/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.6466 - loss: 0.7720
Epoch 10/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 20s 35ms/step - accuracy: 0.6350 - loss: 0.7713


In [70]:
# حفظ النموذج بعد التدريب
hybrid_model.save('hybrid_model.h5')

# لاحقًا يمكن تحميله بهذه الطريقة
from tensorflow.keras.models import load_model
hybrid_model = load_model('hybrid_model.h5')


In [71]:
from tensorflow.keras.models import load_model # تحميل ال hybrid
hybrid_model = load_model('hybrid_model.h5')


In [72]:
# الأعمدة التي تم التدريب عليها
features_model = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg','koi_srad','koi_kepmag']


In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Example values for a new planet, including the engineered features
new_tabular_data = {
    'koi_period': [10.5],
    'koi_duration': [0.12],
    'koi_prad': [1.2],
    'koi_steff': [5800],
    'koi_slogg': [4.4],
    'koi_srad': [1.0],
    'koi_kepmag': [12.3],
    'radius_ratio': [1.2],        # Include this feature
    'orbital_density': [0.1143]   # Include this feature
}

X_tab_new = pd.DataFrame(new_tabular_data)

# Use all 9 features that the scaler was fitted on and transform as a NumPy array
features_for_scaling = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg','koi_srad','koi_kepmag', 'radius_ratio', 'orbital_density']
X_tab_new_scaled = scaler_tab.transform(X_tab_new[features_for_scaling].values)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [76]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# مثال منحنى ضوء جديد (flux)
new_flux = [[1.001, 0.999, 1.002, 1.000, 0.998]]  # يمكن أن يكون أي طول

# توحيد طول السلاسل
MAX_LEN = 200
X_lc_new = pad_sequences(new_flux, maxlen=MAX_LEN, dtype='float32', padding='post', truncating='post')

# تطبيع المنحنيات
X_lc_new = (X_lc_new - X_lc_new.mean(axis=1, keepdims=True)) / (X_lc_new.std(axis=1, keepdims=True) + 1e-6)


In [77]:
# التنبؤ
hybrid_pred_new = hybrid_model.predict([X_tab_new_scaled, X_lc_new])

# تحويل one-hot إلى فئة نصية
pred_class_idx = np.argmax(hybrid_pred_new, axis=1)
pred_class_label = le.inverse_transform(pred_class_idx)

print("Predicted Class:", pred_class_label[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted Class: FALSE POSITIVE


In [79]:
import pandas as pd
import numpy as np

# ==============================
# 1️⃣ قائمة الميزات التي استخدمها النموذج
# ==============================
features_for_scaling = ['koi_period','koi_duration','koi_prad','koi_steff','koi_slogg',
                        'koi_srad','koi_kepmag', 'radius_ratio', 'orbital_density']

# ==============================
# 2️⃣ حساب متوسط القيم لكل تصنيف
# ==============================
confirmed_avg = tabular_df[tabular_df['koi_disposition']=='CONFIRMED'][features_for_scaling].mean()
candidate_avg = tabular_df[tabular_df['koi_disposition']=='CANDIDATE'][features_for_scaling].mean()

# ==============================
# 3️⃣ إنشاء DataFrame جديد لكل مثال
# ==============================
new_confirmed = pd.DataFrame([confirmed_avg])
new_candidate = pd.DataFrame([candidate_avg])

# ==============================
# 4️⃣ تطبيع البيانات بنفس Scaler المستخدم للنموذج
# ==============================
new_confirmed_scaled = scaler_tab.transform(new_confirmed)
new_candidate_scaled = scaler_tab.transform(new_candidate)

# ==============================
# 5️⃣ التنبؤ باستخدام النموذج
# ==============================
pred_conf = best_model.predict(new_confirmed_scaled)
pred_cand = best_model.predict(new_candidate_scaled)

pred_label_conf = le.inverse_transform(np.argmax(pred_conf, axis=1))
pred_label_cand = le.inverse_transform(np.argmax(pred_cand, axis=1))

print("Predicted Class for Confirmed-like planet:", pred_label_conf[0])
print("Predicted Class for Candidate-like planet:", pred_label_cand[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Class for Confirmed-like planet: CONFIRMED
Predicted Class for Candidate-like planet: FALSE POSITIVE


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [80]:
# حفظ النموذج في ملف HDF5
hybrid_model.save('hybrid_model.h5')

print("Model saved successfully as 'hybrid_model.h5'")


Model saved successfully as 'hybrid_model.h5'


In [81]:
import joblib

# حفظ السكالر
joblib.dump(scaler_tab, 'scaler_tab.save')

# حفظ الـ LabelEncoder
joblib.dump(le, 'label_encoder.save')

# تحميلهم لاحقًا
scaler_tab = joblib.load('scaler_tab.save')
le = joblib.load('label_encoder.save')


In [82]:
# 9️⃣ حفظ النماذج
xgb_model.save_model("xgb_physical.json")
lc_model.save("lstm_cnn_lightcurve.h5")
joblib.dump(scaler_tab, "scaler_tab.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']